In [4]:
import gzip
import numpy as np
import os
import os.path as osp
import random
import time

import pandas as pd
import rdflib as rdf

import time
import numpy as np

from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.inspection import permutation_importance

In [5]:
random.seed(0)
np.random.seed(0)

In [ ]:
# import tarfile

# tar_file_path = "linkedmdb-18-05-2009-dump.tar.gz"

# with tarfile.open(tar_file_path, "r:gz") as tar:
#     tar.extractall("extracted_data")

C:\Users\aksha\AppData\Local\Temp\ipykernel_14564\2096243327.py:6: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall("extracted_data")


In [ ]:
# import re


# input_file = "extracted_data/dump.nt"
# output_file = "extracted_data/cleaned_dump.nt"

# with open(input_file, "r", encoding="utf-8") as infile, open(output_file, "w", encoding="utf-8") as outfile:
#     for line in infile:
#         # Skip lines with invalid URIs
#         if '"' in line or "'" in line or "`" in line or "%" in line:
#             continue

#         # Write valid lines to the output file
#         outfile.write(line)

In [8]:
import rdflib

In [9]:


# Path to the extracted RDF file
rdf_file_path = "extracted_data/cleaned_dump.nt"  # Replace with the actual file name

# Parse the RDF file
g = rdflib.Graph()
g.parse(rdf_file_path, format="nt")  # Replace 'turtle' with the correct format  # Update the format if necessary (e.g., 'turtle', 'xml')
len(g)  # This will load the graph into memory

1616709

In [8]:
data = {}

# Populate the dictionary with the RDF triples
for s, p, o in g:
    if str(s) not in data:
        data[str(s)] = {}
    data[str(s)][p] = str(o)
    


# Convert the dictionary to a DataFrame
df = pd.DataFrame.from_dict(data, orient='index')

# Fill NaN values with an empty string for better presentation
df = df.fillna('')
df.index = df.index.astype(str)
len(df)

395556

In [9]:
pd.set_option('display.max_rows', 50)

In [10]:
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)


In [12]:
print(df.head())  # Check the column names
# print(df.dtypes)   # Check the data types of each column

                                          http://data.linkedmdb.org/oddlinker/linkage_run  \
http://data.linkedmdb.org/interlink/68814         http://data.linkedmdb.org/linkage_run/3   
http://data.linkedmdb.org/interlink/47085         http://data.linkedmdb.org/linkage_run/1   
http://data.linkedmdb.org/interlink/68591         http://data.linkedmdb.org/linkage_run/3   
http://data.linkedmdb.org/interlink/28721         http://data.linkedmdb.org/linkage_run/1   
http://data.linkedmdb.org/interlink/67547         http://data.linkedmdb.org/linkage_run/3   

                                          http://www.w3.org/1999/02/22-rdf-syntax-ns#type  \
http://data.linkedmdb.org/interlink/68814   http://data.linkedmdb.org/oddlinker/interlink   
http://data.linkedmdb.org/interlink/47085   http://data.linkedmdb.org/oddlinker/interlink   
http://data.linkedmdb.org/interlink/68591   http://data.linkedmdb.org/oddlinker/interlink   
http://data.linkedmdb.org/interlink/28721   http://data.linkedmdb.org

In [13]:
# Simplify columns and cells
df.columns = df.columns.str.split("#").str[-1]
df.columns = df.columns.str.split("/").str[-1]
df = df.apply(lambda col: col.map(lambda x: x.split("#")[-1]))
df = df.apply(lambda col: col.map(lambda x: x.split("/")[-1]))


In [15]:
for col in df.select_dtypes(include=['object']):
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))

In [17]:
print(df.head())

                                           linkage_run  type  link_source  \
http://data.linkedmdb.org/interlink/68814            3    30        31217   
http://data.linkedmdb.org/interlink/47085            1    30        13183   
http://data.linkedmdb.org/interlink/68591            3    30        19861   
http://data.linkedmdb.org/interlink/28721            1    30        13577   
http://data.linkedmdb.org/interlink/67547            3    30        23282   

                                           link_target  page  made  country  \
http://data.linkedmdb.org/interlink/68814            0     0     0        0   
http://data.linkedmdb.org/interlink/47085            0     0     0        0   
http://data.linkedmdb.org/interlink/68591            0     0     0        0   
http://data.linkedmdb.org/interlink/28721            0     0     0        0   
http://data.linkedmdb.org/interlink/67547            0     0     0        0   

                                           actor  writer  lang

In [20]:
df.head()

,linkage_run,type,link_source,link_target,page,made,country,actor,writer,language,director,film_cut,sameAs,hasPhotoCollection,genre,based_near,music_contributor,producer,editor,cinematographer,film_format,film_collection,SeeAlso,location,sequel,rating,story_contributor,subject,film_series,relatedBook,costume_designer
http://data.linkedmdb.org/interlink/68814,3,30,31217,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
http://data.linkedmdb.org/interlink/47085,1,30,13183,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
http://data.linkedmdb.org/interlink/68591,3,30,19861,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
http://data.linkedmdb.org/interlink/28721,1,30,13577,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
http://data.linkedmdb.org/interlink/67547,3,30,23282,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [24]:
non_zero_columns = (df != 0).sum()
print(non_zero_columns)

linkage_run           162199
type                  395399
link_source           162199
link_target           100603
page                  230488
made                    6922
country                24878
actor                  23555
writer                 14176
language               25961
director               20125
film_cut               19740
sameAs                 25318
hasPhotoCollection     29114
genre                  22095
based_near                 0
music_contributor      10937
producer               13078
editor                  4309
cinematographer         6455
film_format              420
film_collection          439
SeeAlso                12902
location                 391
sequel                  1847
rating                   196
story_contributor        360
subject                  192
film_series              211
relatedBook              700
costume_designer          25
dtype: int64


In [ ]:
# df = pd.get_dummies(df)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif


In [ ]:
X = df
y = np.random.choice(['Label_A', 'Label_B'], size=len(df))  # Example labels (replace with actual labels if available)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Feature selection
# selector = VarianceThreshold(threshold=0.001)
# X_train = selector.fit_transform(X_train)
# X_test = selector.transform(X_test)

selector = SelectKBest(score_func=f_classif, k=20)
X_train = selector.fit_transform(X_train, y_train)
X_test = selector.transform(X_test)

In [27]:
feature_names = df.columns[selector.get_support(indices=True)]


In [2]:
model = GradientBoostingClassifier(n_estimators=500, random_state=0, max_depth=1)
model.fit(X_train, y_train)

NameError: name 'GradientBoostingClassifier' is not defined

In [1]:
accuracy_train = model.score(X_train, y_train)
accuracy_test = model.score(X_test, y_test)
print(f"Model accuracy Training: {accuracy_train}")
print(f"Model accuracy Test: {accuracy_test}")

NameError: name 'model' is not defined

Explainability

In [ ]:
# Explain Model: Permutation feature importance
r = permutation_importance(model, X_test, y_test, n_repeats=5, random_state=0)
importances_mean = r.importances_mean
importances_std = r.importances_std

In [ ]:
top_n = 10
indices = np.argsort(importances_mean)[::-1][:top_n]
top_importances_mean = importances_mean[indices]
top_importances_std = importances_std[indices]

top_feature_names = np.array(feature_names)[indices]
pd.DataFrame({'top_importances_mean': top_importances_mean, 'top_importances_std': top_importances_std}, index=top_feature_names)